# Section II | Stack Visualizing!
---

# Our Question: What factors most accurately predict the score of a question on StackOverflow?

## Part 2: Digging Deeper - More Complex Breakdowns of Scores


1.   number of a views a user has vs score on user's questions
1.   number of a views a user has vs score on user's answers
2. reputation of a user vs score on user's questions
2.   reputation of a user vs score on user's answers
2.   comment length on a question vs score on question
3.   comment length on an answer vs score on answer
1.   num of question badges a user has vs score on user's question
2.   num of answer badges a user has vs score on user's answers
15.  ratio of questions:answers  vs score on user's questions
16. ratio of questions:answers  vs score on user's answers

































### c) Comment Length

#### i) Question Score

In [0]:
%%bigquery --project $project_id q2da1

# YOUR QUERY HERE
SELECT (ROUND(LENGTH(text)/100, 0))*100 AS commLength, AVG(q.score) AS avgqScore
FROM `bigquery-public-data.stackoverflow.comments` c
JOIN `bigquery-public-data.stackoverflow.posts_questions` q
ON q.id = post_id
GROUP BY commLength

In [0]:
%%bigquery --project $project_id q2da2

SELECT 
CASE WHEN MOD(CAST(comm_length AS INT64), 1) = 0 AND comm_length < 3 THEN '<300'
WHEN MOD(CAST(comm_length AS INT64), 1) = 0 AND comm_length < 6 THEN '<600'
ELSE '600+'
END AS
commLength, qScore, countSC/countS AS freq, countS AS numOfInstances
FROM
(SELECT  ROUND (LENGTH(text)/100, 0) AS comm_length, q.score AS qScore, COUNT(*) AS countSC
FROM `bigquery-public-data.stackoverflow.comments` c
JOIN `bigquery-public-data.stackoverflow.posts_questions` q
ON q.id = post_id
GROUP BY comm_length, qScore
) x
JOIN (
SELECT q.score, COUNT(*) AS countS
FROM `bigquery-public-data.stackoverflow.posts_questions` q
JOIN `bigquery-public-data.stackoverflow.comments` c
ON q.id = c.post_id
GROUP BY score 
) y
ON x.qScore = y.score
ORDER BY qScore DESC, commLength DESC

In [0]:
%%bigquery --project $project_id q2da3

SELECT 
CASE WHEN MOD(CAST(comm_length AS INT64), 1) = 0 AND comm_length < 3 THEN '<300'
WHEN MOD(CAST(comm_length AS INT64), 1) = 0 AND comm_length < 6 THEN '<600'
ELSE '600+'
END AS
commLength, qScore, countSC/countS AS freq, countS AS numOfInstances
FROM
(SELECT  ROUND (LENGTH(text)/100, 0) AS comm_length, q.score AS qScore, COUNT(*) AS countSC
FROM `bigquery-public-data.stackoverflow.comments` c
JOIN `bigquery-public-data.stackoverflow.posts_questions` q
ON q.id = post_id
GROUP BY comm_length, qScore
) x
JOIN (
SELECT q.score, COUNT(*) AS countS
FROM `bigquery-public-data.stackoverflow.posts_questions` q
JOIN `bigquery-public-data.stackoverflow.comments` c
ON q.id = c.post_id
GROUP BY score 
) y
ON x.qScore = y.score
WHERE qScore < 5000
ORDER BY qScore DESC, commLength DESC

In [0]:
# YOUR PLOT CODE HERE
import altair as alt 
alt.data_transformers.enable('default', max_rows=None)

alt.Chart(q2da1, title="Average Question Score vs. Comment Length").mark_bar().encode(
    x=alt.X('commLength', bin=True, axis=alt.Axis(title='Comment Length')),
    y=alt.Y('avgqScore', axis=alt.Axis(title='Average Question Score')),
)

Chart({
  data:    commLength  avgqScore
  0       200.0   2.642987
  1       300.0   2.760904
  2         0.0   1.560911
  3       600.0   3.834991
  4       400.0   2.884509
  5       500.0   3.148678
  6       100.0   2.233495,
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Comment Length'
      }),
      bin: True,
      shorthand: 'commLength'
    }),
    y: Y({
      axis: Axis({
        title: 'Average Question Score'
      }),
      shorthand: 'avgqScore'
    })
  }),
  mark: 'bar',
  title: 'Average Question Score vs. Comment Length'
})

In [0]:
import altair as alt
alt.data_transformers.enable('default', max_rows=None)

alt.Chart(q2da2, title="Proportions of Comment Length vs. Question Score Range").mark_bar().encode(
    x=alt.X('sum(freq)', stack="normalize", axis=alt.Axis(title='Proportions of Comment Length')),
    y=alt.Y('qScore', bin=True, axis=alt.Axis(title='Question Score Range')),
    color=alt.Color('commLength', sort=('0','<300', '<600', '600+')),
    tooltip=['commLength', 'freq', 'qScore', 'numOfInstances']
).interactive()

Chart({
  data:      commLength  qScore      freq  numOfInstances
  0          <600   21775  0.142857              14
  1          <600   21775  0.214286              14
  2          <300   21775  0.357143              14
  3          <300   21775  0.142857              14
  4          <300   21775  0.071429              14
  5          600+   21775  0.071429              14
  6          <600   17970  0.125000               8
  7          <600   17970  0.125000               8
  8          <300   17970  0.250000               8
  9          <300   17970  0.500000               8
  10         <300   13957  1.000000               2
  11         <600   10219  0.166667               6
  12         <300   10219  0.500000               6
  13         <300   10219  0.333333               6
  14         <300    8409  1.000000               1
  15         <600    7818  0.133333              30
  16         <600    7818  0.066667              30
  17         <600    7818  0.033333              30
  18         <300    7818  0.466667              30
  19         <300    7818  0.300000              30
  20         <300    7737  0.416667              12
  21         <300    7737  0.583333              12
  22         <600    7678  1.000000               1
  23         <600    7654  0.111111               9
  24         <300    7654  0.333333               9
  25         <300    7654  0.333333               9
  26         600+    7654  0.222222               9
  27         <600    7524  0.100000              10
  28         <300    7524  0.200000              10
  29         <300    7524  0.200000              10
  ...         ...     ...       ...             ...
  6805       <600     -42  0.066667              15
  6806       <300     -42  0.266667              15
  6807       <300     -42  0.266667              15
  6808       <300     -42  0.333333              15
  6809       <300     -47  0.250000               4
  6810       <300     -47  0.500000               4
  6811       <300     -47  0.250000               4
  6812       <600     -51  0.100000              10
  6813       <300     -51  0.100000              10
  6814       <300     -51  0.700000              10
  6815       600+     -51  0.100000              10
  6816       <600     -52  0.111111               9
  6817       <600     -52  0.222222               9
  6818       <300     -52  0.222222               9
  6819       <300     -52  0.333333               9
  6820       <300     -52  0.111111               9
  6821       <600     -65  0.250000               4
  6822       <300     -65  0.500000               4
  6823       <300     -65  0.250000               4
  6824       <300     -70  0.600000               5
  6825       <300     -70  0.400000               5
  6826       <300     -73  0.666667               3
  6827       <300     -73  0.333333               3
  6828       <600     -76  0.062500              16
  6829       <300     -76  0.062500              16
  6830       <300     -76  0.750000              16
  6831       <300     -76  0.125000              16
  6832       <600     -92  0.333333               6
  6833       <300     -92  0.333333               6
  6834       <300     -92  0.333333               6
  
  [6835 rows x 4 columns],
  encoding: EncodingWithFacet({
    color: Color({
      shorthand: 'commLength',
      sort: ('0', '<300', '<600', '600+')
    }),
    tooltip: [Tooltip({
      shorthand: 'commLength'
    }), Tooltip({
      shorthand: 'freq'
    }), Tooltip({
      shorthand: 'qScore'
    }), Tooltip({
      shorthand: 'numOfInstances'
    })],
    x: X({
      axis: Axis({
        title: 'Proportions of Comment Length'
      }),
      shorthand: 'sum(freq)',
      stack: 'normalize'
    }),
    y: Y({
      axis: Axis({
        title: 'Question Score Range'
      }),
      bin: True,
      shorthand: 'qScore'
    })
  }),
  mark: 'bar',
  selection: SelectionMapping({
    selector062: SelectionDef({
      bind: 'scales',
      encoding

In [0]:
import altair as alt
alt.data_transformers.enable('default', max_rows=None)

alt.Chart(q2da3, title="Proportions of Comment Length vs. Question Score Range for Score < 5000").mark_bar().encode(
    x=alt.X('sum(freq)', stack="normalize", axis=alt.Axis(title='Proportions of Comment Length')),
    y=alt.Y('qScore', bin=True, axis=alt.Axis(title='Question Score Range')),
    color=alt.Color('commLength', sort=('0','<300', '<600', '600+')),
    tooltip=['commLength', 'freq', 'qScore', 'numOfInstances']
).interactive()

Chart({
  data:      commLength  qScore      freq  numOfInstances
  0          <300    4965  1.000000               2
  1          <600    4590  0.333333               6
  2          <300    4590  0.500000               6
  3          <300    4590  0.166667               6
  4          <600    4555  0.272727              11
  5          <600    4555  0.090909              11
  6          <300    4555  0.090909              11
  7          <300    4555  0.545455              11
  8          <300    4523  1.000000               1
  9          <600    4509  0.125000               8
  10         <300    4509  0.500000               8
  11         <300    4509  0.250000               8
  12         600+    4509  0.125000               8
  13         <600    4507  0.153846              13
  14         <600    4507  0.076923              13
  15         <600    4507  0.153846              13
  16         <300    4507  0.538462              13
  17         <300    4507  0.076923              13
  18         <300    4482  0.200000               5
  19         <300    4482  0.400000               5
  20         <300    4482  0.400000               5
  21         <600    4379  0.052632              19
  22         <600    4379  0.105263              19
  23         <300    4379  0.473684              19
  24         <300    4379  0.368421              19
  25         <600    4206  0.142857               7
  26         <300    4206  0.142857               7
  27         <300    4206  0.142857               7
  28         <300    4206  0.571429               7
  29         <600    4154  0.153846              13
  ...         ...     ...       ...             ...
  6710       <600     -42  0.066667              15
  6711       <300     -42  0.333333              15
  6712       <300     -42  0.266667              15
  6713       <300     -42  0.266667              15
  6714       <300     -47  0.500000               4
  6715       <300     -47  0.250000               4
  6716       <300     -47  0.250000               4
  6717       <600     -51  0.100000              10
  6718       <300     -51  0.700000              10
  6719       <300     -51  0.100000              10
  6720       600+     -51  0.100000              10
  6721       <600     -52  0.222222               9
  6722       <600     -52  0.111111               9
  6723       <300     -52  0.111111               9
  6724       <300     -52  0.222222               9
  6725       <300     -52  0.333333               9
  6726       <600     -65  0.250000               4
  6727       <300     -65  0.250000               4
  6728       <300     -65  0.500000               4
  6729       <300     -70  0.600000               5
  6730       <300     -70  0.400000               5
  6731       <300     -73  0.666667               3
  6732       <300     -73  0.333333               3
  6733       <600     -76  0.062500              16
  6734       <300     -76  0.062500              16
  6735       <300     -76  0.750000              16
  6736       <300     -76  0.125000              16
  6737       <600     -92  0.333333               6
  6738       <300     -92  0.333333               6
  6739       <300     -92  0.333333               6
  
  [6740 rows x 4 columns],
  encoding: EncodingWithFacet({
    color: Color({
      shorthand: 'commLength',
      sort: ('0', '<300', '<600', '600+')
    }),
    tooltip: [Tooltip({
      shorthand: 'commLength'
    }), Tooltip({
      shorthand: 'freq'
    }), Tooltip({
      shorthand: 'qScore'
    }), Tooltip({
      shorthand: 'numOfInstances'
    })],
    x: X({
      axis: Axis({
        title: 'Proportions of Comment Length'
      }),
      shorthand: 'sum(freq)',
      stack: 'normalize'
    }),
    y: Y({
      axis: Axis({
        title: 'Question Score Range'
      }),
      bin: True,
      shorthand: 'qScore'
    })
  }),
  mark: 'bar',
  selection: SelectionMapping({
    selector063: SelectionDef({
      bind: 'scales',
      encoding

#### ii) Answer Score

In [0]:
%%bigquery --project $project_id q2db1

# YOUR QUERY HERE
SELECT (ROUND(LENGTH(text)/100, 0))*100 AS commLength, AVG(a.score) AS avgaScore
FROM `bigquery-public-data.stackoverflow.comments` c
JOIN `bigquery-public-data.stackoverflow.posts_answers` a
ON a.id = post_id
GROUP BY commLength

In [0]:
%%bigquery --project $project_id q2db2

SELECT 
CASE WHEN MOD(CAST(comm_length AS INT64), 1) = 0 AND comm_length < 3 THEN '<300'
WHEN MOD(CAST(comm_length AS INT64), 1) = 0 AND comm_length < 6 THEN '<600'
ELSE '600+'
END AS
commLength, aScore, countSC/countS AS freq, countS AS numOfInstances
FROM
(SELECT  ROUND (LENGTH(text)/100, 0) AS comm_length, a.score AS aScore, COUNT(*) AS countSC
FROM `bigquery-public-data.stackoverflow.comments` c
JOIN `bigquery-public-data.stackoverflow.posts_answers` a
ON a.id = post_id
GROUP BY comm_length, aScore
) x
JOIN (
SELECT a.score, COUNT(*) AS countS
FROM `bigquery-public-data.stackoverflow.posts_answers` a
JOIN `bigquery-public-data.stackoverflow.comments` c
ON a.id = c.post_id
GROUP BY score 
) y
ON x.aScore = y.score
ORDER BY aScore DESC, commLength DESC

In [0]:
%%bigquery --project $project_id q2db3

SELECT 
CASE WHEN MOD(CAST(comm_length AS INT64), 1) = 0 AND comm_length < 3 THEN '<300'
WHEN MOD(CAST(comm_length AS INT64), 1) = 0 AND comm_length < 6 THEN '<600'
ELSE '600+'
END AS
commLength, aScore, countSC/countS AS freq, countS AS numOfInstances
FROM
(SELECT  ROUND (LENGTH(text)/100, 0) AS comm_length, a.score AS aScore, COUNT(*) AS countSC
FROM `bigquery-public-data.stackoverflow.comments` c
JOIN `bigquery-public-data.stackoverflow.posts_answers` a
ON a.id = post_id
GROUP BY comm_length, aScore
) x
JOIN (
SELECT a.score, COUNT(*) AS countS
FROM `bigquery-public-data.stackoverflow.posts_answers` a
JOIN `bigquery-public-data.stackoverflow.comments` c
ON a.id = c.post_id
GROUP BY score 
) y
ON x.aScore = y.score
WHERE aScore < 5000
ORDER BY aScore DESC, commLength DESC

In [0]:
# YOUR PLOT CODE HERE
import altair as alt 
alt.data_transformers.enable('default', max_rows=None)

alt.Chart(q2db1, title="Average Answer Score vs. Comment Length").mark_bar().encode(
    x=alt.X('commLength', bin=True, axis=alt.Axis(title='Comment Length')),
    y=alt.Y('avgaScore', axis=alt.Axis(title='Average Answer Score')),
)

Chart({
  data:    commLength  avgaScore
  0       400.0   9.280159
  1       500.0   9.753214
  2       100.0   8.286578
  3       200.0   8.969501
  4         0.0   6.506708
  5       300.0   9.138825
  6       600.0  11.668296,
  encoding: EncodingWithFacet({
    x: X({
      axis: Axis({
        title: 'Comment Length'
      }),
      bin: True,
      shorthand: 'commLength'
    }),
    y: Y({
      axis: Axis({
        title: 'Average Answer Score'
      }),
      shorthand: 'avgaScore'
    })
  }),
  mark: 'bar',
  title: 'Average Answer Score vs. Comment Length'
})

In [0]:
import altair as alt
alt.data_transformers.enable('default', max_rows=None)

alt.Chart(q2db2, title="Proportions of Comment Length vs Answer Score Range").mark_bar().encode(
    x=alt.X('sum(freq)', stack="normalize", axis=alt.Axis(title='Proportions of Comment Length')),
    y=alt.Y('aScore', bin=True, axis=alt.Axis(title='Answer Score Range')),
    color=alt.Color('commLength', sort=('0','<300', '<600', '600+')),
    tooltip=['commLength', 'freq', 'aScore', 'numOfInstances']
).interactive()

Chart({
  data:       commLength  aScore      freq  numOfInstances
  0           <600   28708  0.101266              79
  1           <600   28708  0.227848              79
  2           <600   28708  0.088608              79
  3           <300   28708  0.012658              79
  4           <300   28708  0.177215              79
  5           <300   28708  0.303797              79
  6           600+   28708  0.088608              79
  7           <600   19439  0.035714              28
  8           <600   19439  0.214286              28
  9           <300   19439  0.607143              28
  10          <300   19439  0.142857              28
  11          <600   17401  0.111111              18
  12          <600   17401  0.055556              18
  13          <300   17401  0.388889              18
  14          <300   17401  0.388889              18
  15          <300   17401  0.055556              18
  16          <600   14846  0.193548              31
  17          <600   14846  0.032258              31
  18          <300   14846  0.419355              31
  19          <300   14846  0.096774              31
  20          <300   14846  0.258065              31
  21          <600   13060  0.136364              22
  22          <300   13060  0.136364              22
  23          <300   13060  0.500000              22
  24          <300   13060  0.227273              22
  25          <600   12308  0.250000               8
  26          <300   12308  0.125000               8
  27          <300   12308  0.375000               8
  28          600+   12308  0.250000               8
  29          <600   11602  0.116279              43
  ...          ...     ...       ...             ...
  11498       <300     -47  0.333333               3
  11499       <300     -48  0.600000               5
  11500       <300     -48  0.400000               5
  11501       <300     -50  0.375000               8
  11502       <300     -50  0.125000               8
  11503       <300     -50  0.500000               8
  11504       <300     -51  0.750000               4
  11505       <300     -51  0.250000               4
  11506       <300     -52  0.166667               6
  11507       <300     -52  0.333333               6
  11508       <300     -52  0.500000               6
  11509       <600     -54  0.200000               5
  11510       <300     -54  0.800000               5
  11511       <600     -55  0.090909              11
  11512       <600     -55  0.090909              11
  11513       <300     -55  0.181818              11
  11514       <300     -55  0.090909              11
  11515       <300     -55  0.545455              11
  11516       <600     -61  0.090909              22
  11517       <600     -61  0.227273              22
  11518       <300     -61  0.136364              22
  11519       <300     -61  0.363636              22
  11520       <300     -61  0.136364              22
  11521       600+     -61  0.045455              22
  11522       <300     -68  0.142857               7
  11523       <300     -68  0.142857               7
  11524       <300     -68  0.714286               7
  11525       <300    -108  0.666667               6
  11526       <300    -108  0.166667               6
  11527       <300    -108  0.166667               6
  
  [11528 rows x 4 columns],
  encoding: EncodingWithFacet({
    color: Color({
      shorthand: 'commLength',
      sort: ('0', '<300', '<600', '600+')
    }),
    tooltip: [Tooltip({
      shorthand: 'commLength'
    }), Tooltip({
      shorthand: 'freq'
    }), Tooltip({
      shorthand: 'aScore'
    }), Tooltip({
      shorthand: 'numOfInstances'
    })],
    x: X({
      axis: Axis({
        title: 'Proportions of Comment Length'
      }),
      shorthand: 'sum(freq)',
      stack: 'normalize'
    }),
    y: Y({
      axis: Axis({
        title: 'Answer Score Range'
      }),
      bin: True,
      shorthand: 'aScore'
    })
  }),
  mark: 'bar',
  selection: SelectionMapping({
    sel

In [0]:
import altair as alt
alt.data_transformers.enable('default', max_rows=None)

alt.Chart(q2db3, title="Proportions of Comment Length vs Answer Score Range for Score < 5000").mark_bar().encode(
    x=alt.X('sum(freq)', stack="normalize", axis=alt.Axis(title='Proportions of Comment Length')),
    y=alt.Y('aScore', bin=True, axis=alt.Axis(title='Answer Score Range')),
    color=alt.Color('commLength', sort=('0','<300', '<600', '600+')),
    tooltip=['commLength', 'freq', 'aScore', 'numOfInstances']
).interactive()

Chart({
  data:       commLength  aScore      freq  numOfInstances
  0           <600    4897  0.083333              12
  1           <300    4897  0.166667              12
  2           <300    4897  0.333333              12
  3           <300    4897  0.333333              12
  4           600+    4897  0.083333              12
  5           <600    4841  0.030303              33
  6           <600    4841  0.212121              33
  7           <300    4841  0.303030              33
  8           <300    4841  0.060606              33
  9           <300    4841  0.393939              33
  10          <600    4794  0.037037              27
  11          <600    4794  0.037037              27
  12          <300    4794  0.185185              27
  13          <300    4794  0.333333              27
  14          <300    4794  0.407407              27
  15          <600    4783  0.112676              71
  16          <600    4783  0.169014              71
  17          <600    4783  0.126761              71
  18          <300    4783  0.211268              71
  19          <300    4783  0.211268              71
  20          600+    4783  0.169014              71
  21          <600    4738  0.055556              18
  22          <300    4738  0.222222              18
  23          <300    4738  0.111111              18
  24          <300    4738  0.611111              18
  25          <600    4703  0.111111              36
  26          <600    4703  0.027778              36
  27          <600    4703  0.166667              36
  28          <300    4703  0.222222              36
  29          <300    4703  0.388889              36
  ...          ...     ...       ...             ...
  11277       <300     -47  0.333333               3
  11278       <300     -48  0.600000               5
  11279       <300     -48  0.400000               5
  11280       <300     -50  0.500000               8
  11281       <300     -50  0.125000               8
  11282       <300     -50  0.375000               8
  11283       <300     -51  0.250000               4
  11284       <300     -51  0.750000               4
  11285       <300     -52  0.166667               6
  11286       <300     -52  0.500000               6
  11287       <300     -52  0.333333               6
  11288       <600     -54  0.200000               5
  11289       <300     -54  0.800000               5
  11290       <600     -55  0.090909              11
  11291       <600     -55  0.090909              11
  11292       <300     -55  0.545455              11
  11293       <300     -55  0.181818              11
  11294       <300     -55  0.090909              11
  11295       <600     -61  0.227273              22
  11296       <600     -61  0.090909              22
  11297       <300     -61  0.136364              22
  11298       <300     -61  0.136364              22
  11299       <300     -61  0.363636              22
  11300       600+     -61  0.045455              22
  11301       <300     -68  0.714286               7
  11302       <300     -68  0.142857               7
  11303       <300     -68  0.142857               7
  11304       <300    -108  0.166667               6
  11305       <300    -108  0.666667               6
  11306       <300    -108  0.166667               6
  
  [11307 rows x 4 columns],
  encoding: EncodingWithFacet({
    color: Color({
      shorthand: 'commLength',
      sort: ('0', '<300', '<600', '600+')
    }),
    tooltip: [Tooltip({
      shorthand: 'commLength'
    }), Tooltip({
      shorthand: 'freq'
    }), Tooltip({
      shorthand: 'aScore'
    }), Tooltip({
      shorthand: 'numOfInstances'
    })],
    x: X({
      axis: Axis({
        title: 'Proportions of Comment Length'
      }),
      shorthand: 'sum(freq)',
      stack: 'normalize'
    }),
    y: Y({
      axis: Axis({
        title: 'Answer Score Range'
      }),
      bin: True,
      shorthand: 'aScore'
    })
  }),
  mark: 'bar',
  selection: SelectionMapping({
    sel